In [11]:
import os
import zipfile
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from utils import load_dataset, build_label_dict

In [2]:
# Path setup for saving artifacts
ARTIFACT_DIR = "/content/su_baseline_artifacts"
MODEL_DIR = os.path.join(ARTIFACT_DIR, "models")
FEATURE_DIR = os.path.join(ARTIFACT_DIR, "features")
METRIC_DIR = os.path.join(ARTIFACT_DIR, "metrics")
PLOT_DIR = os.path.join(ARTIFACT_DIR, "plots")

In [3]:
os.makedirs(ARTIFACT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(FEATURE_DIR, exist_ok=True)
os.makedirs(METRIC_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)

In [5]:
# Download Dataset from Kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d sangayb/iemocap
with zipfile.ZipFile("iemocap.zip", 'r') as zip_ref:
    zip_ref.extractall("iemocap_data")

Dataset URL: https://www.kaggle.com/datasets/sangayb/iemocap
License(s): CC0-1.0


In [6]:
dataset_root = "iemocap_data"
sessions = ["Session1", "Session2", "Session3", "Session4", "Session5"]

In [7]:
# Build Label Dictionary
label_dict = {}
for session in sessions:
    eval_path = os.path.join(dataset_root, session, "dialog", "EmoEvaluation")
    session_labels = build_label_dict(eval_path)
    label_dict.update(session_labels)

print(f"Total labeled utterances: {len(label_dict)}")

Total labeled utterances: 7532


In [13]:
X, y, speakers = [], [], []
for session in sessions:
    audio_path = os.path.join(dataset_root, session, "sentences", "wav")
    Xi, yi, si = load_dataset(audio_path, label_dict)
    X.extend(Xi)
    y.extend(yi)
    speakers.extend(si)

X = np.array(X)
y = np.array(y)
speakers = np.array(speakers)

np.save(os.path.join(FEATURE_DIR, "X.npy"), X)
np.save(os.path.join(FEATURE_DIR, "y.npy"), y)
np.save(os.path.join(FEATURE_DIR, "speakers.npy"), speakers)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=speakers
)

# Hyperparameter Tuning

In [14]:
# --- SVM Grid ---
subset_size = min(1000, len(X_train))
X_tune, y_tune = X_train[:subset_size], y_train[:subset_size]

svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto']
}

print("Running SVM hyperparameter tuning on subset...")
svm_grid = GridSearchCV(SVC(probability=True), svm_params, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
svm_grid.fit(X_tune, y_tune)

# Final Training
best_svm_params = svm_grid.best_params_
print(f"Best SVM Params (from subset): {best_svm_params}")

print("Training final SVM model on full training set...")
svm_model = SVC(**best_svm_params, probability=True)
svm_model.fit(X_train, y_train)

# Save model
joblib.dump(svm_model, os.path.join(MODEL_DIR, "svm_model.joblib"))
print("Final SVM model saved.")

Running SVM hyperparameter tuning on subset...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best SVM Params (from subset): {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Training final SVM model on full training set...
Final SVM model saved.


In [ ]:
# --- RF Grid ---
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_model = rf_grid.best_estimator_
joblib.dump(rf_model, os.path.join(MODEL_DIR, "rf_model.joblib"))
print(f"Best RF Params: {rf_grid.best_params_}")

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


In [ ]:
# Evaluate and Save
def evaluate_and_save(model, name):
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    matrix = confusion_matrix(y_test, y_pred)

    joblib.dump(report, os.path.join(METRIC_DIR, f"{name}_report.joblib"))
    np.save(os.path.join(METRIC_DIR, f"{name}_confusion.npy"), matrix)

    plt.figure(figsize=(8,6))
    plt.imshow(matrix, cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.colorbar()
    plt.savefig(os.path.join(PLOT_DIR, f"{name}_confusion.png"))
    plt.close()

evaluate_and_save(svm_model, "SVM")
evaluate_and_save(rf_model, "RandomForest")

# Uploading the artifacts folder to Drive

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
!cp -r /content/su_baseline_artifacts /content/drive/MyDrive/

In [25]:
np.unique(y_train)

array(['ang', 'dis', 'exc', 'fea', 'fru', 'hap', 'neu', 'oth', 'sad',
       'sur'], dtype='<U3')